In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import re
from string import digits
from nltk.corpus import stopwords

train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
testdata = pd.read_csv('//kaggle/input/tweet-sentiment-extraction/test.csv')
submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')
testdata = testdata.applymap(str)
stop_words = set(stopwords.words('english'))


In [ ]:
#train model
def remove_non(chars):
    chars = re.sub(r'http\S+', '', chars)
    chars = re.sub(r'[0-9]+', '', chars)
    chars = re.sub(r'[\w\.-]+@[\w\.-]+', '', chars)
    chars = re.sub(r'www\S+', '', chars)
    return chars

def remove_pos(chars):
    new_chars = []
    pos = ('DT', 'EX', 'CD','FW', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'TO', 'VBD', 'VBG', 'VBN', 'WDT', 'WP', 'WP$', 'WRB')
    for word,tag in chars:
        if tag not in pos:
            new_chars.append(word)
    return new_chars

test = testdata.copy()
test['text'] = test['text'].apply(lambda x: x.lower())
test['text'] = test['text'].apply(lambda x: remove_non(x))
test['text'] = test['text'].apply(lambda x: nltk.word_tokenize(x))
test['text'] = test['text'].apply(lambda x: nltk.pos_tag(x))
test['text'] = test['text'].apply(lambda x: remove_pos(x))
test['text'] = test['text'].apply(lambda x: [item for item in x if item not in stop_words])


In [ ]:
positive = list()
negative = list()
neutral = list()

positivetestset = test[test['sentiment'] == 'positive'].copy()
for wo2 in positivetestset['text']:
    positive.append(wo2)
positive = sum(positive, [])
    
negativetestset = test[test['sentiment'] == 'negative'].copy()
for wo3 in negativetestset['text']:
    negative.append(wo3)
negative = sum(negative, [])
    
neutraltestset = test[test['sentiment'] == 'neutral'].copy()
for wo4 in neutraltestset['text']:
    neutral.append(wo4)
neutral = sum(neutral, [])

In [ ]:
positivecounter = dict()
negativecounter = dict()
neutralcounter = dict()

for wo5 in positive:
    positivecounter[wo5] = positivecounter.get(wo5, 0) +1
for wo6 in negative:
    negativecounter[wo6] = negativecounter.get(wo6, 0) +1
for wo7 in neutral:
    neutralcounter[wo7] = neutralcounter.get(wo7, 0) +1


In [ ]:
print(positivecounter)

In [ ]:
positive = list(set(positive))

In [ ]:
negative = list(set(negative))

In [ ]:
neutral = list(set(neutral))

In [ ]:

# for word8 in positive:
#     if word8 in neutral or word8 in negative:
#         positive.remove(word8)
    
# for word9 in negative:
#     if word9 in neutral or word9 in positive:
#         negative.remove(word9)


In [ ]:
        
def intersec(chars, lst): 
    a = set(chars.lower().split())
    b = set(lst)
    c = a.intersection(b)
    return ' '.join(c)

submission.loc [testdata['sentiment'] == 'positive', 'selected_text'] = testdata['text'].apply(lambda x: intersec(x,positive))
submission.loc [testdata['sentiment'] == 'negative', 'selected_text'] = testdata['text'].apply(lambda x: intersec(x,negative))
submission.loc [testdata['sentiment'] == 'neutral', 'selected_text'] = testdata['text'].apply(lambda x: x)
print(submission.head(40))
submission.to_csv('submission.csv', index = False)